In [1]:
import altair as alt
from coop_finances import *

In [2]:
TAX_RATE = 2.32
ROBBIE_PROPANE = 1135
ROBBIE_ELECTRIC = 1598
ROBBIE_HEATING_OIL = 375




def scenarios(property_cost, interest_rate, insurance, lawyer_fees, clt_value, assessed_value, cords_wood, price_per_cord, maintaince, mobile_home, trailer, propane, electric, oil, loan_duration, bookkeeping, nevins_cabin, storage_shed, per_sq_ft, shipping_container, house, house_size, storage_shed_people, shipping_container_ppl, storage_shed_sq_ft, shed_people, shed_sq_ft):
    r = interest_rate / 12
    def make_scenario(name, upfront_costs, number_people):
        P = sum(upfront_costs.values())
        building_values = sum(v for k, v in upfront_costs.items() if k not in {
            "Lawyer Fees",
            "Purchase"
        }) + 80 * 1000
        return Scenario(
            name,
            monthly_cost={
                "Investors": alt.expr.if_(
                    loan_duration == 0,
                    # Interest only
                    r * P,
                    # Fixed rate repayment
                    # https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula
                    alt.expr.if_(
                        r == 0,
                        P / loan_duration,
                        (r * P) / (1 - (1 + r)**(-loan_duration)),
                    ),
                ),
                "Land Trust": alt.expr.if_(clt_value <= 0, 0, 75),
                "Property Tax": assessed_value * TAX_RATE / 100 / 12,
                "Wood": cords_wood *  price_per_cord / 12,
                "Propane": propane / 12,
                "Electric": electric / 12,
                "Heating Oil": oil / 12,
                "Internet": 99,
                "Maintenance": maintaince * building_values / 12,
                "Insurance": insurance / 12,
                "Bookkeeping": bookkeeping
            },
            upfront_cost=upfront_costs,
            number_people=number_people
        )

    def make_scenarios(*scenarios):
        total_upfront = {}
        total_people = {}
        ss = []
        for name, upfront, people in scenarios:
            total_upfront.update(upfront)
            total_people.update(people)
            ss.append(make_scenario(name, dict(total_upfront), dict(total_people)))
        return ss
    return make_scenarios(
        (
            "1. Purchase property",
            {
                "Lawyer Fees": lawyer_fees,
                "Purchase": property_cost - clt_value,
                "Nevin's Cabin - Labor": 500 * 25,
                "Nevin's Cabin - Materials": 5000,
                "Storage Shed + Painting Studio": alt.expr.if_(
                    storage_shed == "shipping",
                    shipping_container,
                    alt.expr.if_(
                        storage_shed == "build",
                        storage_shed_sq_ft * per_sq_ft,
                        0
                    )
                ),
            }, {
                "Mobile Home": mobile_home,
                "Trailer": trailer,
                "Meditation Cabin": alt.expr.if_(nevins_cabin, 1, 0),
                "Storage Shed + Painting Studio": alt.expr.if_(
                    storage_shed == "shipping",
                    shipping_container_ppl,
                    alt.expr.if_(
                        storage_shed == "build",
                        storage_shed_people,
                        0
                    )
                )
            }
        ),
        (
            "2. Renovate existing sheds",
            {
                "Renovate Sheds": shed_sq_ft * per_sq_ft,
            },
            {
                "Renovated Sheds": shed_people,
            }
        ),
        (
            "3. Build a house (remove mobile home)",
            {
                "House": house_size * per_sq_ft,
            },
            {
                "House": house,
                "Mobile Home": 0
            }
        )
    )

generate_plot(
    scenarios,
    [
        "👇 Drag the sliders to change 🏡 values ❣️",
        "https://github.com/saulshanabrook/coop-finances",
    ],
    # Construction
    # Initial
    mobile_home=Variable("🚌", "# of ppl in mobile home", Range(1, 4, 1), 2, 'O'),
    trailer=Variable("🎬", "# of ppl in studio", Range(0, 4, 1), 1, 'O'),
    nevins_cabin=Variable("🧘", "Meditation Cabin", ([True, False], ("Nevin's Cabin", "No meditation cabin")), True, 's'),
    storage_shed=Variable("🫙", "Storage Shed + Painting Studio", (["shipping", "build", 0], ("Shipping container before sale", "Build before sale", "None")), 0, 's'),
    storage_shed_people=Variable("🫙", "# of ppl in Storage Shed + Painting Studio", Range(0, 3, 1), 2, 'o'),
    storage_shed_sq_ft=Variable("🫙", "Storage Shed + Painting Studio sq Ft", Range(500, 1500, 250), 1000, 's'),
    shipping_container=Variable("🚢", "Shipping Container Cost", Range(2 * 1000, 10 * 1000, 1000), 7 * 1000, '$'),
    shipping_container_ppl=Variable("🚢", "# of ppl in Shipping Container", Range(0, 1, 1), 0, 'O'),


    # Shed
    shed_people=Variable("🛖", "# of ppl in renovated sheds", Range(0, 3, 1), 2, 'o'),
    shed_sq_ft=Variable("🛖", "Renovated sheds sq Ft", Range(250, 750, 250), 500, 's'),

    # House
    house=Variable("🏠", "# of ppl in house", Range(0, 5, 1), 3, 'O'),
    house_size=Variable("🏠", "House sq ft", Range(1000, 2500, 250), 2000, 's'),

    per_sq_ft=Variable("👣", "Construction cost / sq ft", Range(100, 400, 50), 150, '$'),


    # Loan costs 
    interest_rate=Variable("📈", "Investor interest rate", Range(0, 0.10, 0.01), 0.03, '%'),
    loan_duration=Variable("🗓", "Loan Duration", ([5 * 12, 10 * 12, 15 * 12, 30 * 12, 0], ["5 years", "10 years", "15 years", "30 years", "Investment"]), 30 * 12, 's'),
    
    # Upfront costs
    property_cost=Variable("🤑", "Purchase price", Range(110 * 1000, 300 * 1000, 10 * 1000), 150 * 1000, '$'),
    lawyer_fees=Variable("💼", "Lawyer fees and closing costs", Range(2000, 20000, 1000), 10000, '$'),
    clt_value=Variable("🏞", "Land value owned by CLT", Range(0, 150 * 1000, 10000), 0, '$'),

    # Other costs
    maintaince=Variable("🔧", "% of building value saved for maintaince", Range(0.01, 0.05, 0.01), 0.03, '%'),
    assessed_value=Variable("🇺🇸", "Assessed Value (for property taxes)", Range(110 * 1000, 300 * 1000, 5 * 1000), 130 * 1000, '$'),
    bookkeeping=Variable("📔", "Bookkeeping.coop Service Plan", ([230, 460, 1150], ["Extra Small", "Small", "Medium"]), 230, 's'),
    # Utilities
    insurance=Variable("🏡", "Home Insurance (annual)", Range(300, 800, 100), 500, '$'),
    cords_wood=Variable("🪵", "Cords of wood per year", Range(0, 5, 1), 3, '#'),
    price_per_cord=Variable("🔥", "Price per cord wood", Range(0, 580, 10), 190, '$'),
    propane=Variable("🦕", "Propane per year", Range(0, ROBBIE_PROPANE * 2, ROBBIE_PROPANE / 10), ROBBIE_PROPANE, '$'),
    electric=Variable("🔌", "Electric per year", Range(0, ROBBIE_ELECTRIC * 2, ROBBIE_ELECTRIC / 10), ROBBIE_ELECTRIC, '$'),
    oil=Variable("🪔", "Heating oil per year", Range(0, ROBBIE_HEATING_OIL * 2, ROBBIE_HEATING_OIL / 10), ROBBIE_HEATING_OIL, '$')
)